<a href="https://colab.research.google.com/github/versant2612/jnotebooks/blob/main/kgtk/06_kg_network_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kgtk==1.0.1

In [2]:
!echo "deb http://downloads.skewed.de/apt bionic main" >> /etc/apt/sources.list
!apt-key adv --keyserver keys.openpgp.org --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-cairo python3-matplotlib
!apt-get install libcairo2-dev 

Executing: /tmp/apt-key-gpghome.PzjN0cFUV0/gpg.1.sh --keyserver keys.openpgp.org --recv-key 612DEFB798507F25
gpg: key 612DEFB798507F25: "Tiago de Paula Peixoto <tiago@skewed.de>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://downloads.skewed.de/apt bionic InRelease
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:11 https://developer.download.nvidia.com

# Network Analysis Using KGTK and graph-tool

In this part of the tutorial you will learn how to do network analysis on a knowledge graph using KGTK and graph-tool (https://graph-tool.skewed.de/), a library for efficient network analysis on large networks. The typical network analysis workflow first extracts a network of interest from a KG, and then applies network analysis tools to the extracted network. KGTK is useful to extract the network as extraction of networks is often an expensive operation on large KGs. KGTK also provides basic, commonly used network analytics (pagerank, connected components, reachability and paths), and can export the networks into the format required by graph-tool, which supports a wide selection onf sophisticated network analytics algorithms (e.g., community detection).

This tutorial is divided into multiple subsections:
- Computing network metrics on the original graph
- Extracting a network of interest from our KG (the extended family of Arnold Schwarzenegger)
- Computing network analytics on the Arnold family graph
- Creating visualizations of KGs using graph-tools
- Computing connected components of a KG

In [33]:
import os

import math
import numpy as np
import pandas as pd

# from graph_tool.all import *

from IPython.display import display, HTML

from kgtk.configure_kgtk_notebooks import ConfigureKGTK

from kgtk.functions import kgtk, kypher

In [34]:
# Parameters

# Folder on local machine where to create the output and temporary folders
input_path = None

output_path = "/tmp/projects"

project_name = "tutorial-network-analysis"


These are all the files that we have, but I am tempted to just use the `all` file as it helps to keep the tutorial simpler

In [35]:
files = [
    "all",
    "label",
    "alias",
    "description",
    "external_id",
    "monolingualtext",
    "quantity",
    "string",
    "time",
    "item",
    "wikibase_property",
    "qualifiers",
    "datatypes",
    "p279",
    "p279star",
    "p31",
    "in_degree",
    "out_degree",
    "pagerank_directed",
    "pagerank_undirected"
]
ck = ConfigureKGTK(files)
ck.configure_kgtk(input_graph_path=input_path,
                  output_path=output_path,
                  project_name=project_name)

User home: /root
Current dir: /content
KGTK dir: /
Use-cases dir: //use-cases
--2021-11-21 01:32:31--  https://github.com/usc-isi-i2/kgtk-tutorial-files/raw/main/datasets/arnold/all.tsv.gz
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/usc-isi-i2/kgtk-notebooks/raw/main/datasets/arnold/all.tsv.gz [following]
--2021-11-21 01:32:31--  https://github.com/usc-isi-i2/kgtk-notebooks/raw/main/datasets/arnold/all.tsv.gz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/usc-isi-i2/kgtk-notebooks/main/datasets/arnold/all.tsv.gz [following]
--2021-11-21 01:32:31--  https://raw.githubusercontent.com/usc-isi-i2/kgtk-notebooks/main/datasets/arnold/all.tsv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.

In [36]:
ck.print_env_variables()

KGTK_GRAPH_CACHE: /tmp/projects/tutorial-network-analysis/temp.tutorial-network-analysis/wikidata.sqlite3.db
OUT: /tmp/projects/tutorial-network-analysis
USE_CASES_DIR: //use-cases
STORE: /tmp/projects/tutorial-network-analysis/temp.tutorial-network-analysis/wikidata.sqlite3.db
TEMP: /tmp/projects/tutorial-network-analysis/temp.tutorial-network-analysis
EXAMPLES_DIR: //examples
kgtk: kgtk
GRAPH: /root/isi-kgtk-tutorial/input
kypher: kgtk query --graph-cache /tmp/projects/tutorial-network-analysis/temp.tutorial-network-analysis/wikidata.sqlite3.db
KGTK_LABEL_FILE: /root/isi-kgtk-tutorial/input/labels.en.tsv.gz
KGTK_OPTION_DEBUG: false
all: /root/isi-kgtk-tutorial/input/all.tsv.gz
label: /root/isi-kgtk-tutorial/input/labels.en.tsv.gz
alias: /root/isi-kgtk-tutorial/input/aliases.en.tsv.gz
description: /root/isi-kgtk-tutorial/input/descriptions.en.tsv.gz
external_id: /root/isi-kgtk-tutorial/input/claims.external-id.tsv.gz
monolingualtext: /root/isi-kgtk-tutorial/input/claims.monolingualtex

Load all my files into the kypher cache so that all graph aliases are defined

In [37]:
%%time
ck.load_files_into_cache()

kgtk query --graph-cache /tmp/projects/tutorial-network-analysis/temp.tutorial-network-analysis/wikidata.sqlite3.db -i "/root/isi-kgtk-tutorial/input/all.tsv.gz" --as all  -i "/root/isi-kgtk-tutorial/input/labels.en.tsv.gz" --as label  -i "/root/isi-kgtk-tutorial/input/aliases.en.tsv.gz" --as alias  -i "/root/isi-kgtk-tutorial/input/descriptions.en.tsv.gz" --as description  -i "/root/isi-kgtk-tutorial/input/claims.external-id.tsv.gz" --as external_id  -i "/root/isi-kgtk-tutorial/input/claims.monolingualtext.tsv.gz" --as monolingualtext  -i "/root/isi-kgtk-tutorial/input/claims.quantity.tsv.gz" --as quantity  -i "/root/isi-kgtk-tutorial/input/claims.string.tsv.gz" --as string  -i "/root/isi-kgtk-tutorial/input/claims.time.tsv.gz" --as time  -i "/root/isi-kgtk-tutorial/input/claims.wikibase-item.tsv.gz" --as item  -i "/root/isi-kgtk-tutorial/input/claims.wikibase-property.tsv.gz" --as wikibase_property  -i "/root/isi-kgtk-tutorial/input/qualifiers.tsv.gz" --as qualifiers  -i "/root/isi-k

# Compute network metrics on the original graph

Pagerank centrality is one of the most useful network metrics as we can use it to rank the results of queries. You can compute pagerank on a KG using the KGTK `graph-statistics` command. We are going to compute pagerank on the KG graph containing items only (no literals or qualifiers) because in our experience we get good results on this subgraph and the algorithm runs faster. 
> You can replace `$item` by `$all` and compute pagerank on the whole graph.

The `graph-statistics` command can also compute in/out degree and hits centrality. In the invocation below we turn off computation of `hits` as it takes longer. You can also specify the properties to be used to construct the graph containing your graph statistics. We are also treating the graph as undirected, as in our experience the pagerank results are more intuitive on the undirected graph:

In [38]:
%%time
kgtk("""
    graph-statistics -i "$item" -o $OUT/metadata.pagerank.undirected.tsv.gz 
    --compute-pagerank True 
    --compute-hits False 
    --page-rank-property Pdirected_pagerank 
    --vertex-in-degree-property Pindegree
    --vertex-out-degree-property Poutdegree
    --output-degrees True 
    --output-pagerank True 
    --output-hits False \
    --output-statistics-only 
    --undirected True 
    --log-file $TEMP/metadata.pagerank.undirected.summary.txt
""")

/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/all.py:39: RuntimeWarning: Error importing draw module, proceeding nevertheless: No module named 'cairo._cairo'
  warnings.warn(msg, RuntimeWarning)

CPU times: user 51.4 ms, sys: 13.6 ms, total: 65 ms
Wall time: 9.58 s


The output is a KG, and is represented using the usual `node1/label/node2/id` TSV file:

In [39]:
kgtk("""
    head -i $OUT/metadata.pagerank.undirected.tsv.gz
""")

,node1,label,node2,id
0,P10,Pindegree,0.000000,P10-Pindegree-0
1,P10,Poutdegree,1.000000,P10-Poutdegree-1
2,P10,Pdirected_pagerank,0.000005,P10-Pdirected_pagerank-2
3,Q18610173,Pindegree,0.000000,Q18610173-Pindegree-3
4,Q18610173,Poutdegree,25.000000,Q18610173-Poutdegree-4
5,Q18610173,Pdirected_pagerank,0.000086,Q18610173-Pdirected_pagerank-5
6,P1000,Pindegree,0.000000,P1000-Pindegree-6
7,P1000,Poutdegree,1.000000,P1000-Poutdegree-7
8,P1000,Pdirected_pagerank,0.000004,P1000-Pdirected_pagerank-8
9,Q18608871,Pindegree,0.000000,Q18608871-Pindegree-9


Now that we have a graph with pagerank information, we can use it in any query to rank the results by pagerank. 
The following query retrieves all `instance of (P31)` of `human (Q5)` from the `item` graph and uses the `pagerank` graph to rank the results by pagerank:

In [40]:
%%time
kgtk("""
    query -i item -i $OUT/metadata.pagerank.undirected.tsv.gz
        --match '
            item: (human)-[:P31]->(:Q5),
            pagerank: (human)-[:Pdirected_pagerank]->(pagerank)'
        --return 'human as node1, pagerank as node2'
        --order-by 'cast(pagerank, float) desc'
    / add-labels
""")

CPU times: user 85.6 ms, sys: 39.3 ms, total: 125 ms
Wall time: 4.25 s


,node1,node2,node1;label
0,Q22686,0.000135,'Donald Trump'@en
1,Q76,0.000131,'Barack Obama'@en
2,Q11812,0.000113,'Thomas Jefferson'@en
3,Q9916,0.000107,'Dwight D. Eisenhower'@en
4,Q23505,0.000103,'George H. W. Bush'@en
...,...,...,...
13869,Q99850,0.000004,'Hu Yaobang'@en
13870,Q99885632,0.000004,'Jean-Marc Gayraud'@en
13871,Q99887066,0.000004,'Gilbert Narcisse'@en
13872,Q999332,0.000004,'Buck Henry'@en


## Create a network of the extended family of `Arnold Schwarzenegger (Q2685)`
We will use the following properties to build the network:
- `child (P40)`
- `sibling (P3373)`
- `spouse (P26)`
- `unmarried partner (P451)`
- `father (P22)`
- `mother (P25)`

And we will exclude:
- `relative (P1038)` (exclude)

Approach:
- Starting from `Arnold Schwarzenegger (Q2685)` find all the nodes that can be reached using the properties listed above
- Extract from the KG all the edges any two nodes computed in the previous step.

> Note: the `reachable-nodes` command only reports nodes, so we have to use the two steps as described above.

The `reachable-nodes` command can compute all the nodes reachable from a set of roots. The roots can be supplied in the command line, and for bulk processing, the root nodes can be specified in a file (it is possible to specify millions of root nodes).

In [41]:
%%time
kgtk("""
    reachable-nodes -i $item
        --root Q2685
        --props P40 P3373 P26 P451 P22 P25
        --label Pextended_family
    / add-labels
""")

CPU times: user 17.9 ms, sys: 7.38 ms, total: 25.3 ms
Wall time: 2.41 s


,node1,label,node2,node1;label,node2;label
0,Q2685,Pextended_family,Q3288486,'Arnold Schwarzenegger'@en,'Gustav Schwarzenegger'@en
1,Q2685,Pextended_family,Q23800370,'Arnold Schwarzenegger'@en,'Aurelia Schwarzenegger'@en
2,Q2685,Pextended_family,Q96076900,'Arnold Schwarzenegger'@en,'Jadrny'@en
3,Q2685,Pextended_family,Q38196234,'Arnold Schwarzenegger'@en,'Meinhard Schwarzenegger'@en
4,Q2685,Pextended_family,Q24004771,'Arnold Schwarzenegger'@en,'Patrick M. Knapp Schwarzenegger'@en
...,...,...,...,...,...
67,Q2685,Pextended_family,Q43100988,'Arnold Schwarzenegger'@en,'Jack Pratt'@en
68,Q2685,Pextended_family,Q901541,'Arnold Schwarzenegger'@en,'Patrick Schwarzenegger'@en
69,Q2685,Pextended_family,Q23800185,'Arnold Schwarzenegger'@en,'Joseph Baena'@en
70,Q2685,Pextended_family,Q75494768,'Arnold Schwarzenegger'@en,'Mildred Patricia Baena'@en


The extended family of Arnold Schwarzenegger in the tutorial KG contains 71 people. 
We store the nodes as a new graph so that it can be used in other KGTK commands:

In [42]:
%%time
kgtk("""
    reachable-nodes -i $item
        --root Q2685
        --props P40 P3373 P26 P451 P22 P25
        --label Pextended_family
        --selflink True
    -o $TEMP/arnold_family_members.tsv
""")

CPU times: user 9.07 ms, sys: 8.07 ms, total: 17.1 ms
Wall time: 1.57 s


In step 2 we extract the edges that connect nodes in the set of reachable nodes. The `where` clause excludes the `relative (P1038)` property:

In [43]:
kgtk("""
    query -i item -i $TEMP/arnold_family_members.tsv
        --match '
            item: (n1)-[l {label: property}]->(n2),
            arnold: ()-[]->(n1),
            arnold: ()-[]->(n2)'
        --where 'property != "P1038"'
        --return 'distinct n1 as node1, property as label, n2 as node2'
    / add-labels
""")

,node1,label,node2,node1;label,label;label,node2;label
0,Q1086823,P22,Q345517,'Christopher Lawford'@en,'father'@en,'Peter Lawford'@en
1,Q1086823,P25,Q432694,'Christopher Lawford'@en,'mother'@en,'Patricia Kennedy Lawford'@en
2,Q1086823,P26,Q75326809,'Christopher Lawford'@en,'spouse'@en,'Jean Edith Olssen'@en
3,Q1086823,P3373,Q75326777,'Christopher Lawford'@en,'sibling'@en,'Victoria Lawford'@en
4,Q1086823,P3373,Q75326779,'Christopher Lawford'@en,'sibling'@en,'Sydney Lawford'@en
...,...,...,...,...,...,...
489,Q9696,P40,Q230303,'John F. Kennedy'@en,'child'@en,'Caroline Kennedy'@en
490,Q9696,P40,Q316064,'John F. Kennedy'@en,'child'@en,'John F. Kennedy Jr.'@en
491,Q9696,P40,Q3290402,'John F. Kennedy'@en,'child'@en,'Patrick Bouvier Kennedy'@en
492,Q9696,P40,Q75326753,'John F. Kennedy'@en,'child'@en,'Arabelle Kennedy'@en


Save the Arnold family in graph `arnold_family`:

In [44]:
%%time
kgtk("""
    query -i item -i $TEMP/arnold_family_members.tsv
        --match '
            item: (n1)-[l {label: property}]->(n2),
            arnold: ()-[]->(n1),
            arnold: ()-[]->(n2)'
        --where 'property != "P1038"'
        --return 'distinct n1 as node1, property as label, n2 as node2'
    / add-id --id-style wikidata
    -o $OUT/arnold.family.tsv
""")


CPU times: user 14.3 ms, sys: 15.6 ms, total: 29.9 ms
Wall time: 1.65 s


In [65]:

kgtk("query -i $OUT/arnold.family.tsv --as arnold_family --limit 10")

,node1,label,node2,id
0,Q1086823,P22,Q345517,Q1086823-P22-Q345517
1,Q1086823,P25,Q432694,Q1086823-P25-Q432694
2,Q1086823,P26,Q75326809,Q1086823-P26-Q75326809
3,Q1086823,P3373,Q75326777,Q1086823-P3373-Q75326777
4,Q1086823,P3373,Q75326779,Q1086823-P3373-Q75326779
5,Q1086823,P3373,Q75326780,Q1086823-P3373-Q75326780
6,Q1086823,P3448,Q96079835,Q1086823-P3448-Q96079835
7,Q1086823,P3448,Q96079836,Q1086823-P3448-Q96079836
8,Q1086823,P3448,Q96079838,Q1086823-P3448-Q96079838
9,Q1086823,P40,Q76363382,Q1086823-P40-Q76363382


### Summary of this section:
In this section:
- We illustrated how to extract a network from a KG focusing on specific relations.
- We built a KG with the family memebers of Arnold.

## Do analytics on the Arnold family graph

In this part of the tutorial we show how to use KGTK to find paths between nodes. We will use the family network we built in the previous step. We specify the source and destination nodes for the paths also as a graph, which in general can be the output of other KGTK commands. Here we built a custom file to provide a few source and destination nodes: 
> We need a nicer way to provide a portable way to define the `path-query` file so that folks can edit it.

In [45]:
%%bash
cat <<EOF >$TEMP/path-query.tsv
node1	node2	label
Q2685	Q4616	path
Q9696	Q230303	path
Q75326777	Q4616	path
Q5112377	Q38196234	path
EOF

Add labels to the path file to confirm that our input is what we want:

In [46]:
kgtk("""
    add-labels -i $TEMP/path-query.tsv
""")

,node1,node2,label,node1;label,node2;label
0,Q2685,Q4616,path,'Arnold Schwarzenegger'@en,'Marilyn Monroe'@en
1,Q9696,Q230303,path,'John F. Kennedy'@en,'Caroline Kennedy'@en
2,Q75326777,Q4616,path,'Victoria Lawford'@en,'Marilyn Monroe'@en
3,Q5112377,Q38196234,path,'Christopher G. Kennedy'@en,'Meinhard Schwarzenegger'@en


We invoke the `paths` command to find the shortest paths between `node1` and `node2`:

In [62]:
%%time
kgtk("""
    paths -i $OUT/arnold.family.tsv
        --verbose False
        --shortest-path True
        --statistics-only True
        --path-file $TEMP/path-query.tsv
    -o $TEMP/path-results.tsv
""")

/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/all.py:39: RuntimeWarning: Error importing draw module, proceeding nevertheless: No module named 'cairo._cairo'
  warnings.warn(msg, RuntimeWarning)

CPU times: user 8.62 ms, sys: 5.63 ms, total: 14.2 ms
Wall time: 1.02 s


In [71]:
kgtk("query -i $TEMP/path-results.tsv --as path-result --limit 10")

,node1,label,node2,id
0,p0,0,Q2685-P26-Q230654,p0-0-0
1,p0,1,Q230654-P25-Q272908,p0-1-1
2,p0,2,Q272908-P3373-Q9696,p0-2-2
3,p0,3,Q9696-P451-Q4616,p0-3-3
4,p1,0,Q9696-P40-Q230303,p1-0-4
5,p2,0,Q75326777-P25-Q432694,p2-0-5
6,p2,1,Q432694-P3373-Q9696,p2-1-6
7,p2,2,Q9696-P451-Q4616,p2-2-7
8,p3,0,Q5112377-P22-Q25310,p3-0-8
9,p3,1,Q25310-P3373-Q272908,p3-1-9


In [72]:
kgtk("query -i $OUT/arnold.family.tsv --as arnold_family --limit 10")

,node1,label,node2,id
0,Q1086823,P22,Q345517,Q1086823-P22-Q345517
1,Q1086823,P25,Q432694,Q1086823-P25-Q432694
2,Q1086823,P26,Q75326809,Q1086823-P26-Q75326809
3,Q1086823,P3373,Q75326777,Q1086823-P3373-Q75326777
4,Q1086823,P3373,Q75326779,Q1086823-P3373-Q75326779
5,Q1086823,P3373,Q75326780,Q1086823-P3373-Q75326780
6,Q1086823,P3448,Q96079835,Q1086823-P3448-Q96079835
7,Q1086823,P3448,Q96079836,Q1086823-P3448-Q96079836
8,Q1086823,P3448,Q96079838,Q1086823-P3448-Q96079838
9,Q1086823,P40,Q76363382,Q1086823-P40-Q76363382


In the output from the `paths` command, `node1` contains an identifier for a path, `node2` contains the edge identifier in the input graph, and `label` contains an integer to order the edges in the path.

The next command uses a query to retrieve the `node1, label, node2` for each of the edges in the paths. We pipe the results through `add-labels` to help us interpret the results.

In [73]:
kgtk("""
    query -i arnold_family -i $TEMP/path-results.tsv
        --match '
            path: (path)-[segment]->(edge),
            arnold_family: (n1)-[edge {label: property}]->(n2)'
        --return 'n1 as node1, property as label, n2 as node2, path as path, segment as segment, edge as edge'
        --order-by 'path, segment'
    / add-labels
""")

,node1,label,node2,path,segment,edge,node1;label,label;label,node2;label
0,Q2685,P26,Q230654,p0,p0-0-0,Q2685-P26-Q230654,'Arnold Schwarzenegger'@en,'spouse'@en,'Maria Shriver'@en
1,Q230654,P25,Q272908,p0,p0-1-1,Q230654-P25-Q272908,'Maria Shriver'@en,'mother'@en,'Eunice Kennedy Shriver'@en
2,Q272908,P3373,Q9696,p0,p0-2-2,Q272908-P3373-Q9696,'Eunice Kennedy Shriver'@en,'sibling'@en,'John F. Kennedy'@en
3,Q9696,P451,Q4616,p0,p0-3-3,Q9696-P451-Q4616,'John F. Kennedy'@en,'unmarried partner'@en,'Marilyn Monroe'@en
4,Q9696,P40,Q230303,p1,p1-0-4,Q9696-P40-Q230303,'John F. Kennedy'@en,'child'@en,'Caroline Kennedy'@en
5,Q75326777,P25,Q432694,p2,p2-0-5,Q75326777-P25-Q432694,'Victoria Lawford'@en,'mother'@en,'Patricia Kennedy Lawford'@en
6,Q432694,P3373,Q9696,p2,p2-1-6,Q432694-P3373-Q9696,'Patricia Kennedy Lawford'@en,'sibling'@en,'John F. Kennedy'@en
7,Q9696,P451,Q4616,p2,p2-2-7,Q9696-P451-Q4616,'John F. Kennedy'@en,'unmarried partner'@en,'Marilyn Monroe'@en
8,Q5112377,P22,Q25310,p3,p3-0-8,Q5112377-P22-Q25310,'Christopher G. Kennedy'@en,'father'@en,'Robert F. Kennedy'@en
9,Q25310,P3373,Q272908,p3,p3-1-9,Q25310-P3373-Q272908,'Robert F. Kennedy'@en,'sibling'@en,'Eunice Kennedy Shriver'@en


### Visualization and graph analytics of the Arnold family graph

The KGTK `export-gt` command makes it easy to export KGs to `graph tool` (https://graph-tool.skewed.de) , a high perfromance graph analytics and visualization tool:

In [74]:
!kgtk export-gt -i $OUT/arnold.family.tsv -o $OUT/arnold.family.gt

/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/all.py:39: RuntimeWarning: Error importing draw module, proceeding nevertheless: No module named 'cairo._cairo'
  warnings.warn(msg, RuntimeWarning)


Once the graph is in `graph-tool` format, it is easy to compute networks metrics such as betweenees centrality, and to visualize the graph.
Below is a visualization of Arnold's family graph where the color of the nodes and the thickness of the lines shows betweenees centrality:

**NOTE** Google Colab VM cant draw these graphs, DO NOT run the following cell

In [51]:
path = os.environ["OUT"] + "/arnold.family.gt"
g = load_graph(path)

bv, be = betweenness(g)
be.a /= be.a.max() / 5
# graph_draw(g, vertex_fill_color=bv, edge_pen_width=be, vertex_size=15)

`graph-tool` is a very rich library. Here is a visualization of the hierarchical structure of Arnold's extended family:
> More work with the library is needed to interpret the visualization (not part of this tutorial)

**NOTE** Google Colab VM cant draw these graphs, DO NOT run the following cell

In [52]:
path = os.environ["OUT"] + "/arnold.family.gt"
g = load_graph(path)

state = minimize_nested_blockmodel_dl(g)
# draw_hierarchy(state)

### Summary of this section:
In this section we:
- Uses the `paths` command to find paths between pairs of nodes.
- Used the `export-gt` command to export a KG to `graph tool` format.
- Used `graph tool` to compute betweeness centrality and visualize a KG.

## Computing connected components
In this section we will find the connected components of the tutorial graph and clean up the graph to remove isolated small components.

The KGTK command `connected-components` finds the connected components of a graph, and outputs a new graph that records the connected component that each item belongs to.

In [53]:
kgtk("""
    connected-components -i $item
        --minimum-cluster-size 1
        --undirected 
        -o $TEMP/item.commected_components.tsv
""")

The `connected-components` command outputs the connected components as a graph:

In [54]:
!head $TEMP/item.commected_components.tsv

node1	label	node2
Q616342	connected_component	CLUS4XI69VtenTCbHUKSPaVsaQ==
Q7560691	connected_component	CLUS4XI69VtenTCbHUKSPaVsaQ==
Q11445559	connected_component	CLUSAHfoFvXvhcEZd73lj1z4Bw==
Q11524722	connected_component	CLUSAHfoFvXvhcEZd73lj1z4Bw==
Q91096397	connected_component	CLUSC4L7DB9+AA0px63bXGxYJA==
Q91097216	connected_component	CLUSC4L7DB9+AA0px63bXGxYJA==
Q78607259	connected_component	CLUSCv46LgpNynxBUfoaweda7w==
Q78607418	connected_component	CLUSCv46LgpNynxBUfoaweda7w==
Q17239842	connected_component	CLUSETikQPaem8mOa44EYJ4hWQ==


Find the sizes of each of the connected components using an aggregation query:

In [77]:
kgtk("""
    query -i $TEMP/item.commected_components.tsv
        --match '
            (item)-[]->(component)'
        --return 'distinct component as node1, count(distinct item) as node2'
        --order-by 'node2 desc'
""")

,node1,node2
0,CLUSTxKFlmIC8ytzvqvCYP7kSA==,66341
1,CLUS4XI69VtenTCbHUKSPaVsaQ==,2
2,CLUSAHfoFvXvhcEZd73lj1z4Bw==,2
3,CLUSC4L7DB9+AA0px63bXGxYJA==,2
4,CLUSCv46LgpNynxBUfoaweda7w==,2
5,CLUSETikQPaem8mOa44EYJ4hWQ==,2
6,CLUSMsI+YaR2hce5HnpP/rq9TQ==,2
7,CLUSNf+5YYA4pq2TcKQzVUbL4A==,2
8,CLUSWhhjZiPd9C8l80phkFJu6Q==,2
9,CLUSiSfL49aiC4iMlarD0N2ONg==,2


There is one large cluster and several small isolated clusters. We want to keep the large cluster, so we select the nodes that belong to the large cluster:

Copiar o ID acima da componente conexa de maior tamanho

In [78]:
kgtk("""
    query -i $TEMP/item.commected_components.tsv
        --match '
            (item)-[]->(:`CLUSTxKFlmIC8ytzvqvCYP7kSA==`)'
        --return 'distinct item as node1'
    / add-labels
""")

,node1,node1;label
0,P10,'video'@en
1,P1000,'record held'@en
2,P1001,'applies to jurisdiction'@en
3,P1004,'MusicBrainz place ID'@en
4,P1005,'Portuguese National Library ID'@en
...,...,...
66336,Q999726,'Dale Van Every'@en
66337,Q999734,'Albert Lasker Award for Basic Medical Researc...
66338,Q99975457,'Faculty of Law of the Károli Gáspár Universit...
66339,Q999763,'University of Paris 1 Pantheon-Sorbonne'@en


Store the large cluster graph in a file (`large-cluster.tsv`) so that we can use it in queries:
> In this example we don't give the graph an alias; instead, we use the path where the graph is stored as an input to the `query` command. KGTK will load the graph in the cache database, and reuse it in subsequent queries:

In [79]:
kgtk("""
    query -i $TEMP/item.commected_components.tsv
        --match '
            (item)-[]->(:`CLUSTxKFlmIC8ytzvqvCYP7kSA==`)'
        --return 'distinct item as node1'
    -o $TEMP/large-cluster.tsv
""")

Now we want to subset the `item` graph to keep the edges that connect nodes in the large cluster, and discard the edges in the small clusters.
We do this in a query that selects all edges from the `item` graph, as long as both nodes in the edge are in the `node1` position in the `large-cluster.tsv` graph.
> Note that the `large-cluster` graph consists of a set of nodes stored in `node1`, and the graph has no edges. The `query` command can use graphs with no edges using the shorthand `cluster: (n1)`

In [80]:
%%time
kgtk("""
    query -i $TEMP/large-cluster.tsv -i item
        --match '
            item: (n1)-[l]->(n2),
            cluster: (n1),
            cluster: (n2)'
        --return 'distinct n1 as node1, l.label as label, n2 as node2, l as id'
""")

CPU times: user 1.67 s, sys: 1.26 s, total: 2.93 s
Wall time: 8.07 s


,node1,label,node2,id
0,P10,P31,Q18610173,P10-P31-Q18610173-85ef4d24-0
1,P1000,P31,Q18608871,P1000-P31-Q18608871-093affb5-0
2,P1001,P1855,Q11696,P1001-P1855-Q11696-cdbf391b-0
3,P1001,P1855,Q12371988,P1001-P1855-Q12371988-12c10bc0-0
4,P1001,P1855,Q181574,P1001-P1855-Q181574-7f428c9b-0
...,...,...,...,...
394306,Q999763,P463,Q868940,Q999763-P463-Q868940-d7659dce-0
394307,Q99979573,P131,Q47164,Q99979573-P131-Q47164-c6e86b16-0
394308,Q99979573,P17,Q30,Q99979573-P17-Q30-aa56b597-0
394309,Q99979573,P30,Q49,Q99979573-P30-Q49-ac39cd28-0


Compare the number of edges of our new graph consisting of the large connected component with the number of edges in the original graph. 
We use the Unix `wc` command to count the number of edges in the original `item` graph:

In [59]:
!zcat< $item | wc -l

394324


The original graph had a few more edges, so looks like our query did the job. Store the large cluster graph in a file so that we can visualize it:

In [84]:
%%time
kgtk("""
    query -i $TEMP/large-cluster.tsv -i item
        --match '
            item: (n1)-[l]->(n2),
            cluster: (n1),
            cluster: (n2)'
        --return 'distinct n1 as node1, l.label as label, n2 as node2, l as id'
    / compact
    -o $OUT/item.large-cluster.tsv
""")

CPU times: user 38 ms, sys: 16.7 ms, total: 54.7 ms
Wall time: 8.33 s


In [85]:
!cat $OUT/item.large-cluster.tsv | wc -l

394312


### Visualization of the large connected component

To compute the visualization, use the `export-gt` to export the large connected component to `graph-tool format`:

In [61]:
!kgtk export-gt -i $OUT/item.large-cluster.tsv -o $OUT/item.large-cluster.gt

/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:32: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/all.py:39: RuntimeWarning: Error importing draw module, proceeding nevertheless: No module named 'cairo._cairo'
  warnings.warn(msg, RuntimeWarning)


The following code loads the graph in `graph-tool` and computes the visualization. It takes 3 minutes to compute the layout and render the visualization, so the code is commented out. An image of the visualization appears below.

```
%%time
path = os.environ["OUT"] + "/item.large-cluster.gt"
g = load_graph(path)

pos = sfdp_layout(g)
graph_draw(g, pos=pos, output_size=(1000, 1000))
```

Visualization of the large connected component of the tutorial graph (393,704 edges). The visualization illustrates the process we used to build the graph: we started at the `Arnold Schwarzenegger (Q2685)` and did a breath first expansion for 3 levels. Then we did a depth-first starting from the nodes representing classes to find the complete closure of the classes.

![Visualization of the tutorial graph](https://github.com/usc-isi-i2/kgtk-tutorial-files/blob/main/media/arnold-graph-vsualization.png?raw=true)

> The following code computes betweeness centrality of the large connected component and visualizes it

```
%%time
path = os.environ["OUT"] + "/item.large-cluster.gt"
g = load_graph(path)

bv, be = betweenness(g)
be.a /= be.a.max() / 5
graph_draw(g, vertex_fill_color=bv, edge_pen_width=be, output_size=(1500, 1500))
```

### Summary of this section
In this section we:
- Computed the connected components of the tutorial graph using the `connected-components` command.
- Discovered that the tutorial graph contained one large cluster (expected) and a few isolated tiny clusters (unexpected).
- Created a new graph discarding the tiny isolated clusters.
- Visualized the large connected component.